### Clinical Drift Detection

In [ ]:
import datetime

from drift_detector.clinical_applicator import ClinicalShiftApplicator
from drift_detector.detector import Detector
from drift_detector.experimenter import Experimenter
from drift_detector.plotter import plot_drift_samples_pval
from drift_detector.reductor import Reductor
from drift_detector.tester import DCTester, TSTester
from gemini.query import get_gemini_data
from gemini.utils import get_label, import_dataset_hospital, normalize, process, scale

## Config Parameters ##

In [ ]:
PATH = "/mnt/nfs/project/delirium/drift_exp/JULY-04-2022/"
HOSPITALS = ["SMH", "MSH", "THPC", "THPM", "UHNTG", "UHNTW", "PMH", "SBK"]
TIMESTEPS = 6
AGGREGATION_TYPE = "time"
ACADEMIC = ["MSH", "PMH", "SMH", "UHNTW", "UHNTG", "PMH", "SBK"]
COMMUNITY = ["THPC", "THPM"]

OUTCOME = input("Select outcome variable: ")
SHIFT = input("Select experiment: ")
MODEL_PATH = (
    "/mnt/nfs/project/delirium/drift_exp/JULY-04-2022/saved_models/"
    + SHIFT
    + "_lstm.pt"
)

if SHIFT == "simulated_deployment":
    exp_params = {
        "source": [datetime.date(2015, 1, 1), datetime.date(2019, 1, 1)],
        "target": [datetime.date(2019, 1, 1), datetime.date(2020, 8, 1)],
        "shift_type": "source_target",
    }

if SHIFT == "covid":
    exp_params = {
        "source": [datetime.date(2019, 1, 1), datetime.date(2020, 2, 1)],
        "target": [datetime.date(2020, 3, 1), datetime.date(2020, 8, 1)],
        "shift_type": "time",
    }

if SHIFT == "seasonal_summer":
    exp_params = {
        "source": [1, 2, 3, 4, 5, 10, 11, 12],
        "target": [6, 7, 8, 9],
        "shift_type": "month",
    }

if SHIFT == "seasonal_winter":
    exp_params = {
        "source": [3, 4, 5, 6, 7, 8, 9, 10],
        "target": [11, 12, 1, 2],
        "shift_type": "month",
    }

if SHIFT == "hosp_type_academic":
    exp_params = {
        "source": ACADEMIC,
        "target": COMMUNITY,
        "shift_type": "hospital_type",
    }

if SHIFT == "hosp_type_community":
    exp_params = {
        "source": COMMUNITY,
        "target": ACADEMIC,
        "shift_type": "hospital_type",
    }

## Query Data ##

In [ ]:
admin_data, x, y = get_gemini_data(PATH)


(X_tr, y_tr), (X_val, y_val), (X_t, y_t), feats, admin_data = import_dataset_hospital(
    admin_data, x, y, SHIFT, OUTCOME, HOSPITALS,
)

x = x.loc[~x.index.get_level_values(0).isin(X_tr.index.get_level_values(0))]

# Normalize training data
X_tr_normalized = normalize(admin_data, X_tr, AGGREGATION_TYPE, TIMESTEPS)
if AGGREGATION_TYPE != "time":
    y_tr = get_label(admin_data, X_tr, OUTCOME)
# Scale training data
X_tr_scaled = scale(X_tr_normalized)
# Process training data
X_tr_final = process(X_tr_scaled, AGGREGATION_TYPE, TIMESTEPS)

## Reductor

In [ ]:
DR_TECHNIQUE = input("Select dimensionality reduction technique: ")

reductor = Reductor(
    dr_method=DR_TECHNIQUE,
    model_path=MODEL_PATH,
    n_features=len(feats),
    var_ret=0.8,
)

## Tester

In [ ]:
TESTER_METHOD = input("Select test method: ")
tstesters = ["lk", "lsdd", "mmd", "tabular", "ctx_mmd", "chi2", "fet", "ks"]
dctesters = ["spot_the_diff", "classifier", "classifier_uncertainty"]

if TESTER_METHOD in tstesters:
    tester = TSTester(
        tester_method=TESTER_METHOD,
    )
elif TESTER_METHOD in dctesters:
    MODEL_METHOD = input("Select model method: ")
    tester = DCTester(
        tester_method=TESTER_METHOD,
        model_method=MODEL_METHOD,
    )

    if MODEL_METHOD == "ctx_mmd":
        CONTEXT_TYPE = input("Select context type: ")

    if MODEL_METHOD == "lk":
        REPRESENTATION = input("Select learned kernel representation: ")

## Detector

In [ ]:
detector = Detector(reductor=reductor, tester=tester, p_val_threshold=0.05)
detector.fit(
    X_tr_final,
    model_path=MODEL_PATH,
    context_type=CONTEXT_TYPE,
    alternative="two-sided",
    n_permutations=100,
)

## ClinicalShiftApplicator

In [ ]:
clinicalshiftapplicator = ClinicalShiftApplicator(shift_type=exp_params["shift_type"])

experimenter = Experimenter(
    detector=detector,
    clinicalshiftapplicator=clinicalshiftapplicator,
    admin_data=admin_data,
)

## Experimenter

In [ ]:
X_val, X_t = experimenter.apply_clinical_shift(
    x, source=exp_params["source"], target=exp_params["target"],
)
# Normalize data
X_val_normalized = normalize(admin_data, X_val, AGGREGATION_TYPE, TIMESTEPS)
X_t_normalized = normalize(admin_data, X_t, AGGREGATION_TYPE, TIMESTEPS)

if AGGREGATION_TYPE != "time":
    y_val = get_label(admin_data, X_val, OUTCOME)
    y_t = get_label(admin_data, X_t, OUTCOME)

# Scale data
X_val_scaled = scale(X_val_normalized)
X_t_scaled = scale(X_t_normalized)

# Process data
X_val_final = process(X_val_scaled, AGGREGATION_TYPE, TIMESTEPS)
X_t_final = process(X_t_scaled, AGGREGATION_TYPE, TIMESTEPS)

val_drift_results = experimenter.detect_shift_samples(
    X_val_final,
    model_path=MODEL_PATH,
    context_type=CONTEXT_TYPE,
    n_permutations=100,
)
test_drift_results = experimenter.detect_shift_samples(
    X_t_final,
    model_path=MODEL_PATH,
    context_type=CONTEXT_TYPE,
    n_permutations=100,
)
shift_results = {"baseline": val_drift_results, "experiment": test_drift_results}

## Plot drift results

In [ ]:
plot_drift_samples_pval(shift_results, 0.05)